## Импорт необходимых библиотек

In [29]:
import re
import spacy
import csv
import torch
import nltk

from itertools import chain
from tqdm import tqdm

## Подготовка данных

In [2]:
nlp = spacy.load('ru_core_news_lg')

In [3]:
data_path = "./data/some_data/processed_data/"

In [4]:
with open(data_path + "original_texts.txt", 'r', encoding='utf-8') as f:
    orig_texts = f.read().split('\n')

In [5]:
with open(data_path + "lemmatized_texts_no_nsubj.txt", 'r', encoding='utf-8') as f:
    lemm_texts_no_nsubj = f.read().split('\n')

In [6]:
pipeline = nlp.pipe_names.copy()
pipeline

['tok2vec', 'morphologizer', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [7]:
pipeline.remove('parser')
pipeline.remove('tok2vec')
pipeline.remove('morphologizer')
pipeline

['ner', 'attribute_ruler', 'lemmatizer']

### Построение csv-файла с необходимыми признаками

In [8]:
total = []

for i, text in tqdm(enumerate((nlp.pipe(orig_texts, disable=pipeline)))):
    for token in text:
        if token.dep_ == 'nsubj':
            morphology = token.morph.to_dict()
            total.append({
                'orig_texts': orig_texts[i],
                'lemm_texts_no_nsubj': lemm_texts_no_nsubj[i],
                'nsubj': token.text,
                'number': morphology.get('Number', 'Sing'),
                'gender': morphology.get('Gender', 'Masc'),
                'verb_tense': token.head.morph.to_dict().get('Tense', 'Pres')
            })
            break

In [275]:
with open(data_path + 'total.csv', 'w', encoding='utf-8') as f:
    csv_writer = csv.writer(f, delimiter=',')
    csv_writer.writerow(['orig_texts', 'lemm_texts_no_nsubj', 'nsubj', 'number', 'gender', 'tense'])
    for item in tqdm(total):
        csv_writer.writerow(item.values())

100%|██████████████████████████████████████████████████████████████████████| 358502/358502 [00:00<00:00, 435706.28it/s]


### Токенизация предложений

In [46]:
tokenized_texts = [nltk.tokenize.wordpunct_tokenize(text.lower()) for text in tqdm(lemm_texts_no_nsubj)]

### Определяем класс словаря, в котором каждому токену ставится в соответствие число

In [44]:
class Vocab:
    def __init__(self, tokens, unk_index):
        self._tokens = tokens
        self._token_to_id = { token: i for i, token in enumerate(tokens) }
        self._unk_index = unk_index
        
    def __len__(self):
        return len(self._tokens)
    
    def word_to_id(self, word):
        return self._token_to_id.get(word, self._unk_index)
    
    def id_to_word(self, idx):
        return self._tokens[idx]